In [1]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Define the language model to use.
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [20]:
# Define the tools to use.
import json
import datetime
from collections import defaultdict
from langchain.agents import tool
from typing import Dict, Text, Any, List
from pydantic.v1 import BaseModel, Field
from langchain.tools.render import format_tool_to_openai_function



@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool(return_direct=False)
def get_peloton_classes() -> str:
    """Get recent Peloton classes."""
    response = json.load(open("peloton_classes.json", "r"))

    today = datetime.datetime.today().date()
    recent_classes = []
    # Create the instructors lookup.
    instructors = {i['id']: i['name'] for i in response['instructors']}
    for w in response['data']:
        workout_date = datetime.datetime.fromtimestamp(w['original_air_time']).date()

        if (today - workout_date).days > 14:
            break

        recent_classes.append(
            {
                'id': w['id'],
                'description': w['description'],
                'difficulty': w['difficulty_estimate'],
                'duration': w['duration'],
                'instructor': instructors[w['instructor_id']],
                'title': w['title'],
                'disciplie': w['fitness_discipline_display_name']
            }
        )
    return json.dumps(recent_classes)


@tool(return_direct=False)
def get_recent_user_workouts() -> str:
    """Get the user's Peloton workouts from the past week."""
    response = json.load(open("user_workouts.json", "r"))

    today = datetime.datetime.today().date()
    recent_workouts = defaultdict(list)
    for w in response['data']:
        workout_date = datetime.datetime.fromtimestamp(w['created_at']).date()

        # Only get workouts from the last 7 days.
        if (today - workout_date).days > 14:
            break

        if 'ride' in w:
            title = w['ride']['title']
        elif 'peloton' in w:
            title = w['peloton']['ride']['title']
        else:
            title = "Unknown"
            
        lbl = f"{workout_date}: {title}"

        recent_workouts[str(workout_date)].append(lbl)

    return json.dumps(recent_workouts)


class StackInput(BaseModel):
    recommended_workout: List[Dict[Text, Any]] = Field(description="the list of recommended classes.")


@tool
def add_class_to_stack(recommended_workout: str):
    """Allows a user to add selected workout to the Peloton stack if the user explicitly asks to.

    recommended_workout: The recommended workout for the user with the class ID for each class in the workout.
    """
    print(recommended_workout)


class RecommendInput(BaseModel):
    user_input: str = Field(description="the user message")


@tool
def recommend_workout(user_workouts: str):
    """Recommend a class for the user."""
    print("USER INPUT")
    print(user_workouts)
    return "This is your recommended class."


tools = [get_word_length, get_recent_user_workouts, get_peloton_classes, add_class_to_stack]



In [21]:
from typing import List, Optional

from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field


class Response(BaseModel):
    """Final response returned to the user."""

    answer: str = Field(description="The final answer to respond to the user")
    raw_data: Optional[Dict[Text, Any]] = Field(
        description="Raw JSON response from the tool if one is provided."
    )

# tools.append(convert_pydantic_to_openai_function(Response))
# tools

In [22]:
# Tell the LLM about the tools available via `bind`.
from langchain.tools.render import format_tool_to_openai_function

openai_tools = [format_tool_to_openai_function(t) for t in tools]
# openai_tools.append(convert_pydantic_to_openai_function(Response))

llm_with_tools = llm.bind(functions=openai_tools)

In [23]:
from langchain.prompts import MessagesPlaceholder, ChatPromptTemplate

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a well qualified personal trainer helping the user to achieve their fitness goals. Chat with the user remembering the following about the user's preferences: the user prefers moderate to high difficulty strength and cyling classes. They do not like barre and pilates. Workouts can consist of 1 or more classes. Unless otherwise instructed the user likes workouts to be 30 minutes long.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [24]:
import json

from langchain.schema.agent import AgentActionMessageLog, AgentFinish

def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    print(name)
    
    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

In [25]:
# Define the agent.
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.agents import AgentExecutor

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [26]:
from langchain.agents import AgentExecutor

chat_history = []
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    return_intermediate_steps=True, 
    handle_parsing_errors="Check your output and make sure it conforms!"
)

In [27]:
input1 = "What should my workout be today?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)

# print(result)



> Entering new AgentExecutor chain...

Invoking: `get_peloton_classes` with `{}`


[{"id": "f5bf0720d7bc43a5ae0da0b17cfda975", "description": "Pick up the pace and get your body moving to an energizing 80s playlist. ", "difficulty": 6.564971751412429, "duration": 2700, "instructor": "Kirsten Ferguson", "title": "45 min 80s Walk", "disciplie": "Walking"}, {"id": "c5eee5d5c0374089bdb782c385e66b9b", "description": "Ride along to some of the best tracks of the 2010s. ", "difficulty": 6.992063492063492, "duration": 1200, "instructor": "Hannah Corbin", "title": "20 min 2010s Ride", "disciplie": "Cycling"}, {"id": "19be447cbee54cacb5fd380092002514", "description": "Bursts of effort followed by recovery, this ride will challenge your mind and strengthen your body inside and out.", "difficulty": 7.583333333333333, "duration": 1200, "instructor": "Hannah Corbin", "title": "20 min Intervals Ride", "disciplie": "Cycling"}, {"id": "9fc58e9c122545238cfdc461603dfee5", "description": "Split your wor

In [28]:
agent_executor.invoke({"input": "Add option 2 to my stack", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_peloton_classes` with `{}`


[{"id": "f5bf0720d7bc43a5ae0da0b17cfda975", "description": "Pick up the pace and get your body moving to an energizing 80s playlist. ", "difficulty": 6.564971751412429, "duration": 2700, "instructor": "Kirsten Ferguson", "title": "45 min 80s Walk", "disciplie": "Walking"}, {"id": "c5eee5d5c0374089bdb782c385e66b9b", "description": "Ride along to some of the best tracks of the 2010s. ", "difficulty": 6.992063492063492, "duration": 1200, "instructor": "Hannah Corbin", "title": "20 min 2010s Ride", "disciplie": "Cycling"}, {"id": "19be447cbee54cacb5fd380092002514", "description": "Bursts of effort followed by recovery, this ride will challenge your mind and strengthen your body inside and out.", "difficulty": 7.583333333333333, "duration": 1200, "instructor": "Hannah Corbin", "title": "20 min Intervals Ride", "disciplie": "Cycling"}, {"id": "9fc58e9c122545238cfdc461603dfee5", "description": "Split your wor

{'input': 'Add option 2 to my stack',
 'chat_history': [HumanMessage(content='What should my workout be today?'),
  AIMessage(content='Based on your preferences for moderate to high difficulty strength and cycling classes, I have a couple of recommendations for today\'s workout. Since you like your workouts to be around 30 minutes long, here are two options:\n\n1. **Cycling Focus:**\n   - Start with "20 min Intervals Ride" with Hannah Corbin. This class has bursts of effort followed by recovery and will challenge your mind and body. (Difficulty: 7.58)\n   - Follow up with "10 min HIIT Cardio" with Callie Gullickson to get a quick, high-intensity cardio session. (Difficulty: 6.47)\n\n2. **Strength and Cycling Combination:**\n   - Begin with "20 min Íconos Latinos Glutes & Legs" with Rad Lopez for a focused strength session on your lower body. (Difficulty: 6.95)\n   - Finish with "10 min HIIT Cardio" with Callie Gullickson for a burst of high-intensity cardio. (Difficulty: 6.47)\n\nBoth 